In [ ]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt



In [ ]:
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

trade_active = mt5.initialize()

print(trade_active)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4200
True


In [ ]:
if not trade_active:
    print('Initialization failed, check internet connection. You must have Meta Trader 5 installed.')
    mt5.shutdown()

else:
    print(mt5.account_info()._asdict())
    print("\n")
    print(mt5.terminal_info()._asdict())
    print("\n")
    print(mt5.symbols_total())



{'login': 213644473, 'trade_mode': 0, 'leverage': 1000, 'limit_orders': 500, 'margin_so_mode': 0, 'trade_allowed': True, 'trade_expert': True, 'margin_mode': 2, 'currency_digits': 2, 'fifo_close': False, 'balance': 200.0, 'credit': 0.0, 'profit': 0.0, 'equity': 200.0, 'margin': 0.0, 'margin_free': 200.0, 'margin_level': 0.0, 'margin_so_call': 25.0, 'margin_so_so': 15.0, 'margin_initial': 0.0, 'margin_maintenance': 0.0, 'assets': 0.0, 'liabilities': 0.0, 'commission_blocked': 0.0, 'name': 'micheal bright omage ', 'server': 'OctaFX-Demo', 'currency': 'USD', 'company': 'Octa Markets Incorporated'}


{'community_account': False, 'community_connection': False, 'connected': True, 'dlls_allowed': False, 'trade_allowed': True, 'tradeapi_disabled': False, 'email_enabled': False, 'ftp_enabled': False, 'notifications_enabled': False, 'mqid': False, 'build': 5200, 'maxbars': 100000, 'codepage': 0, 'ping_last': 150601, 'community_balance': 0.0, 'retransmission': 0.6890690676895515, 'company': 'Meta

In [ ]:
account = mt5.account_info()
terminal = mt5.terminal_info()

print(account.equity)

if(terminal.connected == True and terminal.trade_allowed == True):
    print("AI is successfully functional")
else:
    print("Please make sure metatrade 5 has internet and algo Trade is Turn On")

200.0
AI is successfully functional


In [ ]:
symbols = mt5.symbols_get()
print(len(symbols))
t_symbol = ["GBPUSD","EURUSD","USDCAD","USDJPY","AUDUSD"]
for symbol in symbols:
    for t_s in t_symbol:
        if(symbol.name == t_s):
            print("yes ",t_s)
            break



280
yes  EURUSD
yes  GBPUSD
yes  USDJPY
yes  AUDUSD
yes  USDCAD


In [ ]:
n = 0

In [ ]:
market = mt5.copy_rates_from_pos(t_symbol[n], mt5.TIMEFRAME_M30, 0, 99999)

In [ ]:
market

array([(1503012600, 1.28688, 1.28692, 1.28629, 1.28644, 1504,  8, 0),
       (1503014400, 1.28668, 1.28693, 1.28656, 1.2868 ,  183, 13, 0),
       (1503016200, 1.28677, 1.28694, 1.28643, 1.28664, 2022, 24, 0), ...,
       (1756506600, 1.35073, 1.35148, 1.35073, 1.35147, 1963, 13, 0),
       (1756508400, 1.35148, 1.35164, 1.35101, 1.35124, 1228, 13, 0),
       (1756510200, 1.35125, 1.35126, 1.34995, 1.35018, 2122, 13, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [ ]:
len(market)

99999

In [ ]:
data = []
for i in range(len(market)):
    data.append([market[i][1],market[i][2],market[i][3],market[i][4]])

In [ ]:
df = pd.DataFrame(data, columns=["open", "high","low", "close"])
display(df)

,open,high,low,close
0,1.28688,1.28692,1.28629,1.28644
1,1.28668,1.28693,1.28656,1.28680
2,1.28677,1.28694,1.28643,1.28664
3,1.28667,1.28741,1.28647,1.28671
4,1.28672,1.28736,1.28669,1.28711
...,...,...,...,...
99994,1.35005,1.35105,1.35003,1.35095
99995,1.35093,1.35114,1.35050,1.35073
99996,1.35073,1.35148,1.35073,1.35147
99997,1.35148,1.35164,1.35101,1.35124


In [ ]:
df.to_csv("Generated"+t_symbol[n]+" dbot.csv", index=False)